In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

# CAMBIARLE EL NOMBRE EL SCRIPT A prediccion_intermedia

In [2]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

In [2]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01_undersampled_10_24M.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [3]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005, 202006,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105] # dejo afuera 202106 para test

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(391550, 679)

In [4]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [5]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [6]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


In [7]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Entrenamiento

Cargamos el study de optuna que optimizamos en el script anterior

In [8]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "competencia2_lgbm_v05" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-17 14:34:48,684] Using an existing study with name 'competencia2_lgbm_v05' instead of creating a new one.


In [9]:
resultados = study.trials_dataframe()
resultados.shape

(101, 13)

### Generamos 5 columnas con las predicciones para cada semilla

Usamos cross-validation para que las nuevas columnas no esten distorsionadas por el entrenamiento

In [ ]:
# Pre training to add predictions of the 5 seeds

from sklearn.model_selection import StratifiedKFold

version = 'v007' # UPDATE

# Initialize an empty array to hold the out-of-fold predictions for each model (seed)
predictions = np.zeros((X_train.shape[0], len(semillas)))

# Set up cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for idx, semilla in enumerate(semillas):
    
    best_iter = study.best_trial.user_attrs["best_iter"]
    print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
    
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': study.best_trial.params['num_leaves'],
        'learning_rate': study.best_trial.params['learning_rate'],
        'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
        'feature_fraction': study.best_trial.params['feature_fraction'],
        'bagging_fraction': study.best_trial.params['bagging_fraction'],
        'seed': semilla,
        'verbose': 0
    }
    
    # Loop through each fold in the StratifiedKFold
    for fold_idx, (train_index, val_index) in enumerate(kf.split(X_train, y_train_binaria)):
        
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train_binaria.iloc[train_index], y_train_binaria.iloc[val_index]
    
        w_train_fold = w_train.iloc[train_index]
        w_val_fold = w_train.iloc[val_index]

        # Create LightGBM Dataset
        train_data = lgb.Dataset(X_train_fold, label=y_train_fold, weight=w_train_fold)
        val_data = lgb.Dataset(X_val_fold, label=y_val_fold, weight=w_val_fold, reference=train_data)
        
        # Train the model
        model = lgb.train(params,
                          train_data,
                          num_boost_round=best_iter)
        
        # Predict on the validation fold and store it in the oof_preds matrix
        predictions[val_index, idx] = model.predict(X_val_fold, num_iteration=model.best_iteration)

# At the end of the loop, oof_preds contains the predictions for each fold and seed
# You can now add these predictions as columns in your original dataframe

X_train_with_preds = X_train.copy()
for i, semilla in enumerate(semillas):
    X_train_with_preds[f'pred_s{semilla}'] = predictions[:, i]

# Now, X_train_with_preds has the original data plus the predictions of each fold for each seed


Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996


In [ ]:
X_train_with_preds.shape

(391550, 682)

Para generar las columnas en test no vamos a poder hacer cross validation por lo que entrenamos los modelos intermedios para levantarlos en el script de test y generar las 5 columnas de predicciones, luego sobre ese dataset es donde aplicamos el predict de los modelos finales que se estiman mas abajo.

In [23]:
version = 'v007' # UPDATE

for semilla in semillas:
    
    best_iter = study.best_trial.user_attrs["best_iter"]
    print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
    
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': study.best_trial.params['num_leaves'],
        'learning_rate': study.best_trial.params['learning_rate'],
        'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
        'feature_fraction': study.best_trial.params['feature_fraction'],
        'bagging_fraction': study.best_trial.params['bagging_fraction'],
        'seed': semilla,
        'verbose': 0
    }

    train_data = lgb.Dataset(X_train, # Entrenamos ahora si con las 5 nuevas columnas agregadas
                            label=y_train_binaria,
                            weight=w_train)

    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    model.save_model(modelos_path + f'{version}/lgb_competencia2_{version}_s{semilla}_intermediate.txt')

Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996


In [ ]:
ALGO MAS QUE SE PODRIA PROBAR ES AGREGAR REZAGOS DE ESTAS PROBABILIDADES, CON LA LOGICA DE QUE SI UN CLIENTE TIENE ALTAS PROBABILIDADES EN LOS ULTIMOS MESES EVIDENTEMENTE ESTA AL BORDE
PARA HACER ESO EL PASO ANTERIOR DEBERIA HACERLO CON TODOS LOS DATOS, NO CON EL DATASET UNDERSAMPLEADO.

Generamos y guardamos los modelos que vamos a evaluar en Test

Tenemos entonces los siguientes modelos:

- los usados en cross validation para generar las columnas de prediccion en el set de entrenamiento

- los entrenados con todo el test de entrenamiento (sin las columnas de prediccion), a estos los llamamos intermediate y son los que vamos a usar para generar las columnas de
  prediccion en test ya que ahi no podemos usar la cross validation

- modelo "final" del conjunto de entrenamiento, los que entrenamos en el set de entrenamiento (con las columnas de prediccion), estos son los que vamos a evaluar en test luego de agregar  las columnas de prediccion

- modelo final, volvemos a generar las columnas de prediccion con el dataset de entrenamiento entero (usando cv como en el paso 1) y luego entrenamos los modelos finales sobre el conjunto con las predicciones agregadas.

- tambien generamos los modelos intermediate_final que son los que vamos a usar para generar las columnas de prediccion en el dataset a predecir, y luego para predecir vamos a aplicar el modelo final sobre este nuevo conjunto.

In [22]:
version = 'v007' # UPDATE

for semilla in semillas:
    
    best_iter = study.best_trial.user_attrs["best_iter"]
    print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
    
    params = {
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': study.best_trial.params['num_leaves'],
        'learning_rate': study.best_trial.params['learning_rate'],
        'min_data_in_leaf': study.best_trial.params['min_data_in_leaf'],
        'feature_fraction': study.best_trial.params['feature_fraction'],
        'bagging_fraction': study.best_trial.params['bagging_fraction'],
        'seed': semilla,
        'verbose': 0
    }

    train_data = lgb.Dataset(X_train_with_preds, # Entrenamos ahora si con las 5 nuevas columnas agregadas
                            label=y_train_binaria,
                            weight=w_train)

    model = lgb.train(params,
                    train_data,
                    num_boost_round=best_iter)
    
    model.save_model(modelos_path + f'{version}/lgb_competencia2_{version}_s{semilla}.txt')

Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996
Mejor cantidad de árboles para el mejor model 996
